In [123]:
!pip install lazypredict

In [124]:
__PROJECT_SOURCE__="COLAB"

In [125]:
if __PROJECT_SOURCE__=="COLAB":
    # Import PyDrive and associated libraries.
    # This only needs to be done once per notebook.
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials

    # Authenticate and create the PyDrive client.
    # This only needs to be done once per notebook.
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    from google.colab import drive
    drive.mount('/content/drive')
    fname_dir="/content/blood/"
    fname_url="/content/drive/MyDrive/blood/rework/*agg.csv"
    fname=fname_url.split("/")[-1]
elif __PROJECT_SOURCE__=="LOCAL":
    fname_dir="sdfsdfsd" #working dir
    fname_url="/content/drive/MyDrive/blood/rework/*agg.csv" #data source dir
    


Mounted at /content/drive


In [126]:
!mkdir $fname_dir

mkdir: cannot create directory ‘/content/blood/’: File exists


In [127]:
!cp $fname_url $fname_dir

In [128]:
if __PROJECT_SOURCE__=="COLAB":
    drive.flush_and_unmount()
    print('Unmount Google Drive :-(')

Unmount Google Drive :-(


## Feldolgozás

In [129]:
!rm *.hdf5

rm: cannot remove '*.hdf5': No such file or directory


In [130]:
import pandas as pd

In [131]:
fnames_list=["ldl1_agg.csv","ldl2_agg.csv"]

In [132]:
file_name1=fname_dir+fnames_list[0]
print(file_name1)

/content/blood/ldl1_agg.csv


In [133]:
file_name2=fname_dir+fnames_list[1]
print(file_name2)

/content/blood/ldl2_agg.csv


In [134]:
df_agg1= pd.read_csv(file_name1)
df_agg1.describe()
df_agg1.drop(df_agg1[df_agg1.absorbance0 < 0].index, inplace=True) # kill the negative elements


In [135]:
df_agg2= pd.read_csv(file_name2)
df_agg2.describe()
df_agg2.drop(df_agg2[df_agg2.absorbance0 < 0].index, inplace=True) # kill the negative elements


In [136]:
df_agg=pd.concat([df_agg1,df_agg2], ignore_index=True)

In [137]:
df_agg.tail()

,Unnamed: 0,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,...,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,donation_id,temperature,humidity,cholesterol_ldl_value,cholesterol_ldl_human
537,405,0.56,0.56,0.56,0.57,0.57,0.58,0.58,0.59,0.60,0.61,0.62,0.64,0.64,0.65,0.65,0.65,0.64,0.64,0.64,0.64,0.63,0.63,0.63,0.62,0.62,0.62,0.62,0.62,0.61,0.61,0.61,0.61,0.62,0.62,0.62,0.62,0.63,0.63,0.63,...,1.44,1.43,1.42,1.41,1.40,1.40,1.39,1.38,1.37,1.36,1.35,1.35,1.34,1.33,1.33,1.32,1.32,1.31,1.30,1.30,1.30,1.29,1.29,1.29,1.29,1.28,1.28,1.28,1.27,1.27,1.27,1.26,1.27,1.30,1.27,11886,36.54,65.20,88.54,1
538,406,0.59,0.59,0.59,0.59,0.60,0.60,0.61,0.61,0.62,0.63,0.64,0.65,0.66,0.66,0.66,0.66,0.66,0.65,0.65,0.65,0.64,0.64,0.64,0.63,0.63,0.63,0.63,0.62,0.62,0.62,0.62,0.62,0.62,0.62,0.62,0.63,0.63,0.63,0.64,...,1.46,1.45,1.45,1.44,1.43,1.42,1.41,1.40,1.39,1.38,1.37,1.36,1.36,1.35,1.34,1.33,1.33,1.32,1.32,1.31,1.30,1.30,1.30,1.29,1.29,1.29,1.28,1.28,1.27,1.25,1.24,1.24,1.24,1.25,1.24,11889,39.05,56.01,159.69,3
539,407,0.48,0.48,0.48,0.49,0.50,0.50,0.51,0.51,0.52,0.53,0.54,0.55,0.56,0.56,0.56,0.56,0.56,0.55,0.55,0.55,0.54,0.54,0.54,0.53,0.53,0.53,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.53,0.53,0.53,...,1.49,1.48,1.46,1.45,1.43,1.42,1.40,1.39,1.38,1.37,1.35,1.34,1.33,1.32,1.31,1.30,1.29,1.28,1.28,1.27,1.26,1.26,1.25,1.25,1.25,1.25,1.24,1.24,1.23,1.21,1.18,1.16,1.16,1.17,1.16,11890,46.12,46.85,99.20,1
540,408,0.57,0.56,0.57,0.57,0.58,0.59,0.59,0.60,0.60,0.61,0.63,0.64,0.64,0.65,0.65,0.64,0.64,0.64,0.64,0.63,0.63,0.63,0.62,0.62,0.62,0.62,0.61,0.61,0.61,0.61,0.61,0.61,0.61,0.61,0.61,0.62,0.62,0.62,0.63,...,1.51,1.50,1.49,1.48,1.47,1.46,1.45,1.44,1.43,1.42,1.41,1.40,1.39,1.38,1.38,1.37,1.37,1.36,1.35,1.35,1.34,1.34,1.33,1.33,1.33,1.33,1.32,1.32,1.31,1.29,1.28,1.27,1.26,1.28,1.27,11891,40.38,51.92,164.71,3
541,409,0.46,0.46,0.47,0.47,0.47,0.48,0.48,0.49,0.50,0.51,0.52,0.53,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.53,0.53,0.53,0.53,0.52,0.52,0.51,0.51,0.51,0.51,0.50,0.50,0.50,0.50,0.50,0.50,0.51,0.51,0.51,0.52,...,1.33,1.32,1.31,1.30,1.29,1.28,1.27,1.25,1.25,1.24,1.23,1.22,1.21,1.20,1.20,1.19,1.18,1.17,1.17,1.17,1.16,1.16,1.15,1.15,1.15,1.15,1.15,1.15,1.15,1.15,1.15,1.15,1.16,1.18,1.17,11966,36.49,50.90,98.99,1


In [138]:
df_agg.describe()

,Unnamed: 0,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,...,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,donation_id,temperature,humidity,cholesterol_ldl_value,cholesterol_ldl_human
count,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,...,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00,542.00
mean,170.70,0.52,0.52,0.53,0.53,0.54,0.54,0.55,0.56,0.56,0.57,0.59,0.60,0.61,0.61,0.61,0.61,0.61,0.61,0.60,0.60,0.60,0.59,0.59,0.59,0.58,0.58,0.58,0.57,0.57,0.57,0.57,0.57,0.57,0.57,0.57,0.58,0.58,0.58,0.59,...,1.50,1.49,1.47,1.46,1.45,1.44,1.43,1.42,1.41,1.40,1.39,1.38,1.37,1.36,1.35,1.34,1.34,1.33,1.32,1.32,1.31,1.31,1.30,1.30,1.30,1.30,1.30,1.30,1.30,1.29,1.28,1.28,1.27,1.29,1.28,6927.53,42.11,39.22,111.44,1.49
std,120.51,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,...,0.09,0.09,0.09,0.09,0.09,0.08,0.08,0.08,0.08,0.08,0.08,0.08,0.08,0.08,0.08,0.08,0.08,0.07,0.07,0.07,0.07,0.07,0.07,0.07,0.07,0.07,0.08,0.08,0.08,0.08,0.09,0.09,0.10,0.10,0.10,3094.96,3.42,8.95,44.31,1.03
min,0.00,0.40,0.40,0.41,0.41,0.42,0.42,0.43,0.43,0.44,0.46,0.47,0.48,0.49,0.49,0.49,0.49,0.49,0.49,0.48,0.48,0.48,0.47,0.47,0.47,0.46,0.46,0.46,0.46,0.45,0.45,0.45,0.45,0.45,0.45,0.45,0.46,0.46,0.46,0.47,...,1.20,1.19,1.18,1.18,1.16,1.15,1.15,1.14,1.13,1.12,1.11,1.10,1.10,1.09,1.08,1.08,1.07,1.06,1.06,1.05,1.05,1.04,1.04,1.04,1.04,1.04,1.04,1.04,1.03,1.03,1.02,1.00,0.98,0.97,0.95,1974.00,31.38,16.14,5.30,0.00
25%,68.00,0.49,0.49,0.50,0.50,0.51,0.51,0.52,0.53,0.54,0.55,0.56,0.57,0.58,0.58,0.58,0.58,0.58,0.58,0.58,0.57,0.57,0.57,0.56,0.56,0.56,0.55,0.55,0.55,0.55,0.54,0.54,0.54,0.54,0.54,0.55,0.55,0.55,0.55,0.56,...,1.44,1.43,1.42,1.41,1.40,1.39,1.38,1.37,1.36,1.35,1.34,1.33,1.32,1.31,1.30,1.30,1.29,1.28,1.28,1.27,1.27,1.26,1.26,1.26,1.26,1.25,1.25,1.25,1.25,1.24,1.23,1.22,1.21,1.22,1.21,4150.50,39.42,33.59,81.35,1.00
50%,136.50,0.52,0.52,0.52,0.53,0.53,0.54,0.55,0.55,0.56,0.57,0.59,0.60,0.60,0.61,0.61,0.61,0.61,0.60,0.60,0.60,0.59,0.59,0.59,0.58,0.58,0.58,0.58,0.57,0.57,0.57,0.57,0.57,0.57,0.57,0.57,0.57,0.58,0.58,0.58,...,1.51,1.50,1.48,1.47,1.46,1.45,1.43,1.42,1.41,1.40,1.39,1.38,1.37,1.36,1.35,1.34,1.34,1.33,1.32,1.32,1.31,1.31,1.31,1.30,1.30,1.30,1.30,1.30,1.30,1.29,1.28,1.27,1.27,1.29,1.27,7086.50,41.81,38.47,109.85,1.00
75%,273.75,0.55,0.55,0.55,0.56,0.56,0.57,0.58,0.58,0.59,0.60,0.61,0.63,0.63,0.64,0.64,0.64,0.63,0.63,0.63,0.63,0.62,0.62,0.62,0.61,0.61,0.61,0.60,0.60,0.60,0.60,0.60,0.60,0.60,0.60,0.60,0.60,0.61,0.61,0.61,...,1.56,1.55,1.53,1.52,1.51,1.50,1.48,1.47,1.46,1.45,1.44,1.43,1.4

In [139]:

import tensorflow as tf
from keras.layers import InputLayer, Dense, LSTM, Input, Dropout,Embedding, Flatten,LayerNormalization
from keras.models import Sequential, Model

import keras.optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.backend import clear_session
from tensorflow.keras.losses import mean_absolute_percentage_error, huber,kld,mse

from keras.preprocessing.sequence import pad_sequences
import numpy as np


In [172]:
X_columns=df_agg.columns[1:-5]
y_columns=df_agg.columns[-1] # the category is at the end of columns
X_=df_agg[X_columns]
y_=df_agg[y_columns]

In [173]:
X_columns

Index(['absorbance0', 'absorbance1', 'absorbance2', 'absorbance3',
       'absorbance4', 'absorbance5', 'absorbance6', 'absorbance7',
       'absorbance8', 'absorbance9',
       ...
       'absorbance160', 'absorbance161', 'absorbance162', 'absorbance163',
       'absorbance164', 'absorbance165', 'absorbance166', 'absorbance167',
       'absorbance168', 'absorbance169'],
      dtype='object', length=170)

In [174]:
y_.replace(3,2, inplace=True) # the generated file contains  0,1,3 as the category label :-( 

In [143]:
#y_.replace(2,0, inplace=True)

In [144]:
y_=np.array(y_)

In [145]:
y_=y_.reshape(-1,1)

In [175]:
y_

0      1
1      1
2      1
3      1
4      0
      ..
537    1
538    2
539    1
540    2
541    1
Name: cholesterol_ldl_human, Length: 542, dtype: int64

In [190]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer

X_columns=df_agg.columns[1:-5]
y_columns=df_agg.columns[-1] # the category is at the end of columns
X_=df_agg[X_columns]
y_=df_agg[y_columns]

y_.replace(3,2, inplace=True) # the generated file contains  0,1,3 as the category label :-( 

y_=np.array(y_)
y_=y_.reshape(-1,1)



__SCALER__="normal"

__DNN_MODE__="regression"



if __DNN_MODE__=="regression":
    y_columns=df_agg.columns[-2] # the category is at the  -2  column
    y_=df_agg[y_columns]
    y_=np.array(y_)
    y_=y_.reshape(-1,1)


elif __DNN_MODE__=="classification":
    y_columns=df_agg.columns[-1] # the category is at the end of columns
    y_=df_agg[y_columns]
    y_.replace(3,2, inplace=True) # the generated file contains  0,1,3 as the category label :-( 
    y_=np.array(y_)
    y_=y_.reshape(-1,1)




if __SCALER__=="normal":
    X_normalizer = Normalizer() 
    X=X_normalizer.fit_transform(X_)
    #y_normalizer = Normalizer() 
    #y=y_normalizer.fit_transform(y_)


elif __SCALER__=="minmax":
    X_minmax = MinMaxScaler() 
    X=X_minmax.fit_transform(X_)
    #y_minmax = MinMaxScaler() 
    
    #y=y_minmax.fit_transform(y_)
    




In [191]:
y_

array([[105.6    ],
       [ 72.4    ],
       [ 68.4    ],
       [106.     ],
       [ 31.2    ],
       [ 40.6    ],
       [ 82.6    ],
       [ 32.7    ],
       [ 64.2    ],
       [151.     ],
       [ 94.8    ],
       [ 78.6    ],
       [110.8    ],
       [148.2    ],
       [ 82.6    ],
       [130.2    ],
       [211.6    ],
       [ 52.2    ],
       [ 91.2    ],
       [ 37.6    ],
       [121.6    ],
       [114.4    ],
       [114.6    ],
       [118.2    ],
       [ 78.2    ],
       [102.8    ],
       [188.4    ],
       [137.8    ],
       [134.     ],
       [165.4    ],
       [ 95.2    ],
       [ 68.2    ],
       [157.6    ],
       [157.6    ],
       [ 74.     ],
       [ 79.2    ],
       [147.4    ],
       [119.     ],
       [152.8    ],
       [127.04   ],
       [ 98.     ],
       [ 44.96   ],
       [ 63.18   ],
       [106.62   ],
       [ 85.58   ],
       [110.08   ],
       [ 86.56   ],
       [ 91.54   ],
       [122.28   ],
       [ 47.4    ],


In [149]:
'''
import wandb
from wandb.keras import WandbCallback

wandb.init(config={"hyper": "parameter"})
'''



'\nimport wandb\nfrom wandb.keras import WandbCallback\n\nwandb.init(config={"hyper": "parameter"})\n'

In [150]:
def blood_model_2():
    #clear_session()

    keras.backend.clear_session()

    input_len=171

    print(input_len)
    output_size=24
    drop_frac0=0.0 
    drop_frac1=0.0



    input1=Input(shape=(input_len,))

    #flatt=Flatten()(lstm1)

    non=42
    #initializer = tf.keras.initializers.LecunNormal()
    #initializer=tf.keras.initializers.LecunUniform()
    #initializer=tf.keras.initializers.HeUniform(    seed=None)
    #initializer= tf.keras.initializers.RandomNormal(    mean=3.0, stddev=0.05, seed=None)

    initializer="HeNormal"
    d1=Dense(1000,activation="sigmoid",kernel_initializer=initializer)(input1)
    d1=Dense(100,activation="sigmoid",kernel_initializer=initializer)(d1)
    #d1=Dense(100,activation="selu",kernel_initializer=initializer)(d1)


    #softmax
    pred=Dense(1,activation="sigmoid",)(d1)

    
    
    model = Model(inputs=input1, outputs=pred)

    opt = tf.keras.optimizers.Adamax(learning_rate=0.001)


    lossfn = tf.keras.metrics.SparseCategoricalCrossentropy()

    model.compile(loss="CategoricalCrossentropy",
        optimizer=opt,
        metrics=["Accuracy"])
    return(model)

In [151]:
model_name="_col"
def scheduler(epoch, lr):
    return 0.001

callback_LR = tf.keras.callbacks.LearningRateScheduler(scheduler)

callbacks = [callback_LR,
        
        #savemodela,
        ModelCheckpoint(filepath=model_name+"_{loss:.5f}_{val_loss:.5f}_.hdf5", monitor='val_loss',
                        verbose=1, save_best_only=True, mode='min')]

In [152]:
new_model_2=blood_model_2()

171


In [153]:
new_model_2.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 171)]             0         
                                                                 
 dense (Dense)               (None, 1000)              172000    
                                                                 
 dense_1 (Dense)             (None, 100)               100100    
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 272,201
Trainable params: 272,201
Non-trainable params: 0
_________________________________________________________________


In [154]:
!rm *.hdf5

rm: cannot remove '*.hdf5': No such file or directory


In [155]:
y

array([[1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [2],
       [1],
       [1],
       [1],
       [2],
       [1],
       [2],
       [2],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [2],
       [2],
       [2],
       [2],
       [1],
       [1],
       [2],
       [2],
       [1],
       [1],
       [2],
       [1],
       [2],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [2],
       [1],
       [1],
       [2],
       [2],
       [1],
       [2],
       [1],
       [2],
       [1],
       [1],
       [0],
       [1],
       [2],
       [1],
       [2],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [2],
       [1],
       [1],
       [2],
    

In [156]:
#new_model_2.load_weights("./_col_0.00098_0.00173_.xhdf5")

In [157]:
X

array([[0.03217347, 0.03234261, 0.03258208, ..., 0.087287  , 0.08787816,
        0.08845451],
       [0.03442654, 0.03444344, 0.03470552, ..., 0.08507478, 0.08541812,
        0.08371092],
       [0.03160118, 0.03181977, 0.03219423, ..., 0.08506588, 0.08521799,
        0.08513546],
       ...,
       [0.03215178, 0.03245838, 0.03259315, ..., 0.07791085, 0.07867791,
        0.07812427],
       [0.03763083, 0.03757427, 0.03790216, ..., 0.08403246, 0.08546423,
        0.08446565],
       [0.03494193, 0.03508661, 0.03524443, ..., 0.0878896 , 0.08905506,
        0.08834595]])

In [158]:
'''
history=new_model_2.fit(X,
          y,
          epochs=500, 
          batch_size=3,
          validation_split=0.2,
          verbose=1,
          callbacks=[callbacks],
          shuffle=True
          
          )
'''

'\nhistory=new_model_2.fit(X,\n          y,\n          epochs=500, \n          batch_size=3,\n          validation_split=0.2,\n          verbose=1,\n          callbacks=[callbacks],\n          shuffle=True\n          \n          )\n'

In [159]:
from lazypredict.Supervised import LazyClassifier

In [186]:
y

array([[1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [2],
       [1],
       [1],
       [1],
       [2],
       [1],
       [2],
       [2],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [2],
       [2],
       [2],
       [2],
       [1],
       [1],
       [2],
       [2],
       [1],
       [1],
       [2],
       [1],
       [2],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [2],
       [1],
       [1],
       [2],
       [2],
       [1],
       [2],
       [1],
       [2],
       [1],
       [1],
       [0],
       [1],
       [2],
       [1],
       [2],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [2],
       [1],
       [1],
       [2],
    

In [187]:
clf = LazyClassifier(verbose=1,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X[:-100], X[-100:], y[:-100], y[-100:])

  3%|▎         | 1/29 [00:00<00:11,  2.34it/s]

{'Model': 'AdaBoostClassifier', 'Accuracy': 0.38, 'Balanced Accuracy': 0.28870408870408876, 'ROC AUC': None, 'F1 Score': 0.3488840543047475, 'Time taken': 0.4263288974761963}


  7%|▋         | 2/29 [00:00<00:10,  2.60it/s]

{'Model': 'BaggingClassifier', 'Accuracy': 0.45, 'Balanced Accuracy': 0.3473180873180873, 'ROC AUC': None, 'F1 Score': 0.3858119658119658, 'Time taken': 0.35205602645874023}
{'Model': 'BernoulliNB', 'Accuracy': 0.42, 'Balanced Accuracy': 0.33645183645183646, 'ROC AUC': None, 'F1 Score': 0.40539029869373494, 'Time taken': 0.01728367805480957}


 24%|██▍       | 7/29 [00:02<00:07,  2.91it/s]

...

{'Model': 'RandomForestClassifier', 'Accuracy': 0.51, 'Balanced Accuracy': 0.3753707553707554, 'ROC AUC': None, 'F1 Score': 0.4390823355748729, 'Time taken': 0.4530677795410156}
{'Model': 'RidgeClassifier', 'Accuracy': 0.53, 'Balanced Accuracy': 0.42665280665280664, 'ROC AUC': None, 'F1 Score': 0.4748950402049384, 'Time taken': 0.022303104400634766}
{'Model': 'RidgeClassifierCV', 'Accuracy': 0.51, 'Balanced Accuracy': 0.34234234234234234, 'ROC AUC': None, 'F1 Score': 0.35504415400918404, 'Time taken': 0.04916048049926758}
{'Model': 'SGDClassifier', 'Accuracy': 0.41, 'Balanced Accuracy': 0.35602217602217606, 'ROC AUC': None, 'F1 Score': 0.40280746309088444, 'Time taken': 0.05484771728515625}
{'Model': 'SVC', 'Accuracy': 0.5, 'Balanced Accuracy': 0.3333333333333333, 'ROC AUC': None, 'F1 Score': 0.33333333333333326, 'Time taken': 0.08961248397827148}


 97%|█████████▋| 28/29 [00:06<00:00,  3.86it/s]

{'Model': 'XGBClassifier', 'Accuracy': 0.48, 'Balanced Accuracy': 0.3530284130284131, 'ROC AUC': None, 'F1 Score': 0.41171297884505437, 'Time taken': 1.7482280731201172}


100%|██████████| 29/29 [00:08<00:00,  3.62it/s]

{'Model': 'LGBMClassifier', 'Accuracy': 0.45, 'Balanced Accuracy': 0.31639639639639644, 'ROC AUC': None, 'F1 Score': 0.38545083639330885, 'Time taken': 1.6253044605255127}


In [185]:
print(models)

                               Accuracy  ...  Time Taken
Model                                    ...            
LinearSVC                          0.51  ...        0.58
LinearDiscriminantAnalysis         0.46  ...        0.06
NearestCentroid                    0.39  ...        0.02
RidgeClassifier                    0.53  ...        0.02
LogisticRegression                 0.55  ...        0.09
GaussianNB                         0.30  ...        0.02
Perceptron                         0.40  ...        0.02
RandomForestClassifier             0.51  ...        0.46
DecisionTreeClassifier             0.45  ...        0.09
ExtraTreesClassifier               0.48  ...        0.22
SGDClassifier                      0.41  ...        0.06
XGBClassifier                      0.48  ...        1.87
BaggingClassifier                  0.45  ...        0.35
LabelPropagation                   0.40  ...        0.03
LabelSpreading                     0.40  ...        0.04
KNeighborsClassifier           

In [ ]:
yo=[i[0] for i in y_]
yo

In [ ]:
X

In [200]:
from lazypredict.Supervised import LazyRegressor
reg = LazyRegressor(verbose=2, ignore_warnings=False, custom_metric=None)
models,predictions = clf.fit(X[:-100], X[-100:], yo[:-100], yo[-100:])

100%|██████████| 29/29 [00:00<00:00, 51869.86it/s]


In [182]:
print(models)

Empty DataFrame
Columns: [Accuracy, Balanced Accuracy, ROC AUC, F1 Score, Time Taken]
Index: []


In [221]:
X_train=X[:-100]
X_test=X[-100:]
y_train=y[:-100]
y_test =y[-100:]


In [237]:


from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score


clf = RandomForestClassifier(max_depth=5, random_state=0,n_estimators=3)
clf.fit(X_train, y_train)

randomforest_predict=clf.predict(X_train)

In [238]:
print(f"Accuracy: {accuracy_score(y_train,randomforest_predict)}")

Accuracy: 0.7104072398190046


In [239]:
randomforest_predict=clf.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test,randomforest_predict)}")

Accuracy: 0.51


In [162]:
---

SyntaxError: ignored

In [ ]:
y_pred=new_model_2.predict(X)
result_df=pd.DataFrame()
y_pred_col=[i[0] for i in y_pred]
y_col=[i[0] for i in y]

result_df["y"]=y_col
result_df["y_pred"]=y_pred_col

In [ ]:
import plotly.express as px

fig = px.scatter(result_df, x="y", y="y_pred",width=600, height=400 )
fig.show()